In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
df_competition = spark.read.option("multiline", "true").json("s3://sor/football/data/competitions.json")

In [9]:
df_competition.printSchema

<bound method DataFrame.printSchema of DataFrame[competition_gender: string, competition_id: bigint, competition_international: boolean, competition_name: string, competition_youth: boolean, country_name: string, match_available: string, match_available_360: string, match_updated: string, match_updated_360: string, season_id: bigint, season_name: string]>

In [10]:
display(df_competition)

DataFrame[competition_gender: string, competition_id: bigint, competition_international: boolean, competition_name: string, competition_youth: boolean, country_name: string, match_available: string, match_available_360: string, match_updated: string, match_updated_360: string, season_id: bigint, season_name: string]

In [11]:
df_competition.show()

+------------------+--------------+-------------------------+----------------+-----------------+------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+
|competition_gender|competition_id|competition_international|competition_name|competition_youth|country_name|     match_available|match_available_360|       match_updated|   match_updated_360|season_id|season_name|
+------------------+--------------+-------------------------+----------------+-----------------+------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+
|              male|             9|                    false|   1. Bundesliga|            false|     Germany|2023-12-12T07:43:...|               null|2023-12-12T07:43:...|                null|       27|  2015/2016|
|              male|            16|                    false|Champions League|            false|      Europe|2023-03-07T12:20:...|          

# Explorando os dados 

## Quais competições tem na base ?

In [26]:
df_competition.select("competition_name", "competition_id").distinct().orderBy("competition_name").show(truncate=False)

+-----------------------+--------------+
|competition_name       |competition_id|
+-----------------------+--------------+
|1. Bundesliga          |9             |
|Champions League       |16            |
|Copa del Rey           |87            |
|FA Women's Super League|37            |
|FIFA U20 World Cup     |1470          |
|FIFA World Cup         |43            |
|Indian Super league    |1238          |
|La Liga                |11            |
|Liga Profesional       |81            |
|Ligue 1                |7             |
|Major League Soccer    |44            |
|NWSL                   |49            |
|North American League  |116           |
|Premier League         |2             |
|Serie A                |12            |
|UEFA Euro              |55            |
|UEFA Europa League     |35            |
|UEFA Women's Euro      |53            |
|Women's World Cup      |72            |
+-----------------------+--------------+



## Qual é a edição de Champions League mais recente ?

In [78]:
df_champions = df_competition.select("competition_name", "season_name").where("competition_id = 16").orderBy(df_competition.season_name.desc())
df_champions.show()

+----------------+-----------+
|competition_name|season_name|
+----------------+-----------+
|Champions League|  2018/2019|
|Champions League|  2017/2018|
|Champions League|  2016/2017|
|Champions League|  2015/2016|
|Champions League|  2014/2015|
|Champions League|  2013/2014|
|Champions League|  2012/2013|
|Champions League|  2011/2012|
|Champions League|  2010/2011|
|Champions League|  2009/2010|
|Champions League|  2008/2009|
|Champions League|  2006/2007|
|Champions League|  2004/2005|
|Champions League|  2003/2004|
|Champions League|  1999/2000|
|Champions League|  1972/1973|
|Champions League|  1971/1972|
|Champions League|  1970/1971|
+----------------+-----------+



In [77]:
champions_mais_recente = df_champions.agg({"season_name":"max"}).withColumnRenamed("max(season_name)", "season_mais_recente")

champions_mais_recente.show()


+----------+
|season_max|
+----------+
| 2018/2019|
+----------+

